# Problem 3

## Problem (a)

### 1. Download(Generate) dynamics dataset

In [1]:
# simulating a 1D CLG model ensemble takes a long time
# so will use a pre-computed ensemble of runs
# done on my local machine, which can be downloaded via:
!wget -O clg_finals_dynamics_10000samples.npy https://github.com/kimmw3002/CLP_HW1/raw/refs/heads/main/clg_finals_dynamics_10000samples.npy

--2025-05-07 11:55:37--  https://github.com/kimmw3002/CLP_HW1/raw/refs/heads/main/clg_finals_dynamics_10000samples.npy
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kimmw3002/CLP_HW1/refs/heads/main/clg_finals_dynamics_10000samples.npy [following]
--2025-05-07 11:55:38--  https://raw.githubusercontent.com/kimmw3002/CLP_HW1/refs/heads/main/clg_finals_dynamics_10000samples.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100000128 (95M) [application/octet-stream]
Saving to: ‘clg_finals_dynamics_10000samples.npy’

clg_finals_dynamics 100%[===================>]  95.37M   363MB/s    in 0.3s 

In [ ]:
# clg.py
# on windows, make this a separate .py file to avoid issues with multiprocessing
# can be run directly as a cell in colab

# simulating a 1D CLG model ensemble takes a long time
# so will use a pre-computed ensemble of runs
# done on my local machine, which can be downloaded from the cell above
# in theory, you could run this to generate the ensemble on colab
import numpy as np
import warnings
from concurrent.futures import ProcessPoolExecutor

# ---------- single‑run simulator ----------
def run_CLG_until_absorbing(L, N, max_steps=1_000_000, rng=None):
    rng = rng or np.random.default_rng()
    conf = np.zeros(L, dtype=np.int8)
    conf[rng.choice(L, N, replace=False)] = 1

    for _ in range(max_steps):
        occ = conf == 1
        left_occ   = np.roll(occ,  1)
        right_occ  = np.roll(occ, -1)
        left_emp   = np.roll(conf,  1) == 0
        right_emp  = np.roll(conf, -1) == 0
        movable = np.where(occ & (left_occ | right_occ) & (left_emp | right_emp))[0]
        if movable.size == 0:
            return conf
        i = rng.choice(movable)
        targets = [(i - 1) % L] if left_emp[i] else []
        if right_emp[i]:
            targets.append((i + 1) % L)
        j = rng.choice(targets)
        conf[j], conf[i] = 1, 0
    warnings.warn("max_steps reached before absorbing", RuntimeWarning)
    return conf

# ---------- helper for the pool ----------
def _worker(args):
    L, N, max_steps, seed = args
    return run_CLG_until_absorbing(L, N, max_steps, rng=np.random.default_rng(seed))

# ---------- public ensemble ----------
def run_ensemble_cpu(L, N, runs, max_steps=1_000_000, n_workers=None, seed=None):
    master = np.random.default_rng(seed)
    seeds  = master.integers(0, 2**63 - 1, size=runs)
    args   = [(L, N, max_steps, s) for s in seeds]
    with ProcessPoolExecutor(max_workers=n_workers) as pool:
        finals = list(pool.map(_worker, args, chunksize=1))
    return np.array(finals)            # shape (runs, L)

# ---------- run the ensemble ----------
if __name__ == "__main__":
    L, rho, R = 10_000, 0.4, 10000    # lattice, density, #runs
    finals = run_ensemble_cpu(L, int(L*rho), R, n_workers=None, seed=42)
    np.save("clg_finals_dynamics_10000samples.npy", finals)
    print("CPU ensemble done & saved.")

### 2. Generate combinatorics dataset

In [2]:
import numpy as np

def run_dataset2_fast(L, N, C):
    """
    Returns a (C, L) array of 0/1 rows, each with N ones and no two 1s adjacent.
    """
    M = L - N + 1
    # 1) pick N “slots” in [0..M-1] per row
    rand = np.random.rand(C, M)
    idxs = np.argpartition(rand, N-1, axis=1)[:, :N]
    idxs.sort(axis=1)

    # 2) shift so no two 1’s touch
    offsets   = np.arange(N)
    positions = idxs + offsets  # (C, N)

    # 3) scatter into zero array
    data = np.zeros((C, L), dtype=np.int8)
    rows = np.arange(C)[:, None]
    data[rows, positions] = 1

    return data

L, N, C = 10000, 4000, 10000   # parameters for the dataset
data = run_dataset2_fast(L, N, C)
# save to .npy
np.save("clg_finals_combinatorics_10000samples.npy", data)
print(f"Saved to clg_finals_combinatorics_10000samples.npy")

Saved to clg_finals_combinatorics_10000samples.npy


## Problem (b)

In [3]:
import numpy as np
import zlib

def compress_lza(data_bytes):
    """
    Placeholder LZ77-based compressor wrapper.
    Currently uses zlib (DEFLATE) for fast C-based compression.
    Returns compressed byte-length.
    """
    return len(zlib.compress(data_bytes))

def compute_cid_stats(dataset, compressor):
    """
    Returns (mean_cid, std_cid) for CID = compressed_length / original_length
    computed over each row of `dataset` (shape (C, L)).
    """
    C, L = dataset.shape
    cids = np.empty(C, dtype=float)
    for i in range(C):
        row_bytes   = dataset[i].astype(np.uint8).tobytes()
        compressed  = compressor(row_bytes)
        cids[i]     = compressed / len(row_bytes)
    return cids.mean(), cids.std()

dyn_file  = "clg_finals_dynamics_10000samples.npy"
comb_file = "clg_finals_combinatorics_10000samples.npy"

# load datasets
dynamics      = np.load(dyn_file)
combinatorics = np.load(comb_file)

# compute mean and standard deviation of CID
mean_dyn,  std_dyn  = compute_cid_stats(dynamics,      compress_lza)
mean_comb, std_comb = compute_cid_stats(combinatorics, compress_lza)

print(f"Dynamics CID:      mean = {mean_dyn:.6f}, std = {std_dyn:.6f}")
print(f"Combinatorics CID: mean = {mean_comb:.6f}, std = {std_comb:.6f}")

Dynamics CID:      mean = 0.095458, std = 0.001225
Combinatorics CID: mean = 0.106414, std = 0.000825


1. **Standard error of each mean**

   $$
   \mathrm{SE}_1 = \frac{\sigma_1}{\sqrt{N_1}}
     = \frac{0.001225}{\sqrt{10000}}
     = 1.225\times10^{-5},
   \quad
   \mathrm{SE}_2 = \frac{\sigma_2}{\sqrt{N_2}}
     = \frac{0.000825}{\sqrt{10000}}
     = 8.25\times10^{-6}.
   $$

2. **Standard error of the difference**

   $$
   \mathrm{SE}_\Delta
     = \sqrt{\mathrm{SE}_1^2 + \mathrm{SE}_2^2}
     = \sqrt{(1.225\times10^{-5})^2 + (8.25\times10^{-6})^2}
     \approx 1.477\times10^{-5}.
   $$

3. **Difference of means**

   $$
   \Delta \mu
     = \bar x_2 - \bar x_1
     = 0.106414 - 0.095458
     = 0.010956.
   $$

4. **Z-score**

   $$
   z
     = \frac{\Delta \mu}{\mathrm{SE}_\Delta}
     = \frac{0.010956}{1.477\times10^{-5}}
     \approx 741.8.
   $$

5. **Two-sided \(p\)-value**

   $$
   p
     = 2\,(1 - \Phi(z))
     \approx 0.
   $$

---

**Conclusion:**  
The difference remains hundreds of standard errors apart—overwhelmingly significant. This proves that the dynamically reached ensemble is different from the combinatorics based ensemble generation. The dynamically reached states are not a uniform probability ensemble of all combinatorically possible states.

## Problem (c)

We change the hyperparameter `n_blocks` to test the performance of the 1d CNN. One "block" is composed with `Conv1d` + `ReLU` + `MaxPool1d`. For the first block, the number of channels is selected as 16, and for each another block, the number of channel doubles. Other hyperparameters are the most typically used ones.

In [4]:
"""
Full, self-contained script to compare 1, 2, 3 convolutional “blocks”.

A *block* = Conv1d(kernel_size =3, stride = 1, padding = 1)
         → ReLU
         → MaxPool1d(kernel_size = 2, stride = 2, padding = 0)

Conv channel width doubles after every block.
After n_blocks pools the 1-D length shrinks to  L // (2**n_blocks).

The first Linear therefore needs:
    flat_dim = (base_ch * 2**(n_blocks-1)) * (L // 2**n_blocks)
"""

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# ───────────────────────────────── hyper‑params ──────────────────────────────
L           = 10_000          # input sequence length
base_ch     = 16              # channels of the first conv
batch_size  = 64
epochs      = 10
lr          = 1e-3
block_list  = (1, 2, 3)       # experiment sweep
# ───────────────────────────────── reproducibility ────────────────────────────
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
# ───────────────────────────────────── model ──────────────────────────────────
class CLGModel(nn.Module):
    def __init__(self, L: int, n_blocks: int, base_ch: int = 16):
        super().__init__()

        layers = []
        in_ch, out_ch = 1, base_ch

        for _ in range(n_blocks):
            layers.extend([
                nn.Conv1d(in_channels=in_ch,
                          out_channels=out_ch,
                          kernel_size=3,  # explicit
                          stride=1,       # explicit
                          padding=1),     # explicit
                nn.ReLU(inplace=True),
                nn.MaxPool1d(kernel_size=2,  # explicit
                             stride=2,       # explicit
                             padding=0)      # explicit
            ])
            in_ch, out_ch = out_ch, out_ch * 2  # double channels

        self.conv = nn.Sequential(*layers)

        seq_len_after = L // (2 ** n_blocks)
        channels_after = base_ch * (2 ** (n_blocks - 1))
        flat_dim = channels_after * seq_len_after

        self.cls = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flat_dim, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 2)              # raw logits for CrossEntropyLoss
        )

    def forward(self, x):
        x = x.view(x.size(0), 1, -1)       # (B,1,L)
        x = self.conv(x)
        return self.cls(x)

# ───────────────────────────────────── data ───────────────────────────────────
dynamics       = np.load("clg_finals_dynamics_10000samples.npy")
combinatorics  = np.load("clg_finals_combinatorics_10000samples.npy")

X = np.concatenate((dynamics, combinatorics), axis=0)
y = np.concatenate((np.zeros(len(dynamics)), np.ones(len(combinatorics))), axis=0)

indices = np.random.permutation(len(X))
X, y = X[indices], y[indices]

train_end = int(0.8 * len(X))
val_end   = int(0.9 * len(X))

X_train, y_train = X[:train_end],          y[:train_end]
X_val,   y_val   = X[train_end:val_end],   y[train_end:val_end]
X_test,  y_test  = X[val_end:],            y[val_end:]

def to_loader(Xn, yn, shuffle=False):
    X_t = torch.from_numpy(Xn).float()
    y_t = torch.from_numpy(yn).long()
    ds  = TensorDataset(X_t, y_t)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle, pin_memory=True)

train_loader = to_loader(X_train, y_train, shuffle=True)
val_loader   = to_loader(X_val,   y_val)
test_loader  = to_loader(X_test,  y_test)

device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

# ────────────────────────────── helpers ───────────────────────────────────────
def train_one_epoch(model, loader, optimizer):
    model.train()
    running = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        loss = criterion(model(xb), yb)
        loss.backward()
        optimizer.step()
        running += loss.item() * xb.size(0)
    return running / len(loader.dataset)

@torch.no_grad()
def accuracy(model, loader):
    model.eval()
    correct = tot = 0
    for xb, yb in loader:
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        correct += (model(xb).argmax(1) == yb).sum().item()
        tot     += yb.size(0)
    return correct / tot

# ───────────────────────────── experiment loop ───────────────────────────────
for n_blocks in block_list:
    model = CLGModel(L=L, n_blocks=n_blocks, base_ch=base_ch).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    print(f"\n─── {n_blocks} BLOCK{'S' if n_blocks>1 else ''} ─────────────────────")
    for epoch in range(1, epochs + 1):
        tr_loss = train_one_epoch(model, train_loader, optimizer)
        val_acc = accuracy(model, val_loader)
        print(f"Epoch {epoch:2d} | Train Loss {tr_loss:.4f} | Val Acc {val_acc:.4f}")

    tst_acc = accuracy(model, test_loader)
    print(f"Test Accuracy: {tst_acc:.4f}")



─── 1 BLOCK ─────────────────────
Epoch  1 | Train Loss 1.0421 | Val Acc 0.4975
Epoch  2 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  3 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  4 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  5 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  6 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  7 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  8 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  9 | Train Loss 0.6932 | Val Acc 0.4975
Epoch 10 | Train Loss 0.6932 | Val Acc 0.4975
Test Accuracy: 0.4855

─── 2 BLOCKS ─────────────────────
Epoch  1 | Train Loss 0.8416 | Val Acc 0.4975
Epoch  2 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  3 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  4 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  5 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  6 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  7 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  8 | Train Loss 0.6932 | Val Acc 0.4975
Epoch  9 | Train Loss 0.6932 | Val Acc 0.4975
Epoch 10 | Train Loss 0.6932 | Va

The result is surprising: for `n_blocks` value 1 or 2, the model struggles to learn - the accuracy is close to 50%, which suggests the model is nearly random guessing. But for `n_blocks` value 3, the model suddenly classifies the two datasets to a perfect level! This suggests that the characteristic difference of the two datasets is "long-ranged" so at least 3 blocks of convolution + max pooling is required to extract the feature that discriminates the two datasets.